## 1. Data Scraping from Website: <a name="data"></a>

Two libraries, namely [requests](https://realpython.com/python-requests/) and [BeautifulSoup4](https://beautiful-soup-4.readthedocs.io/en/latest/) are required to scrape and process the data from website, also, the data will be ultimately saved as a local copy with .csv format and hence, the necessary libraries are imported into the notebook:

In [1]:
# Install the beautifulsoup library if they are not installed yet
import sys
!{sys.executable} -m pip install beautifulsoup4

In [2]:
# Install the third party parser (Optional, Python has built-in HTML parser)
!{sys.executable} -m pip install lxml

In [4]:
# Import libraries
from bs4 import BeautifulSoup
import requests # library to handle requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# import csv library 
import csv

print('Libraries imported.')



Libraries imported.


The data is available on the Wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) as given in the assignment.

We will start the scraping of data from assigning the **URL** and getting the content using **requests.get**.

In [5]:
# Assign URL
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# use 'requests' to get the website content
try:
    response = requests.get(url)
    print("Website responded successfully")
except:
    print("Error occured, failed to get data. Please check.")


Website responded successfully


Next, we apply html (or lxml) parser on the result we obtained from the website using **BeautifulSoup** module.

In [6]:
# Apply beautifulsoup module and html parser
soup_html = BeautifulSoup(response.text, 'html.parser')

# Check the title of the file
print(soup_html.text[0:500])





List of postal codes of Canada: M - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xg1lggpAAEMAAHzLN7wAAADJ","wgCSPNonce"


From inspecting the elements of Wikipedia page, the table is found with the *class="wikitable sortable"* as seen in the ![Screenshot of wikipedia page, inspect element](https://raw.githubusercontent.com/ahdelim/IBM_Coursera_Capstone_Course8-9/master/Inspect_element_week3.png). 



Hence we can run the following command to find the table.


In [7]:
# use .find to look for the 'table' from the html file
result_table = soup_html.find('table', class_='wikitable sortable')
result_table.text[0:200]

'\n\nPostcode\nBorough\nNeighborhood\n\n\nM1A\nNot assigned\nNot assigned\n\n\nM2A\nNot assigned\nNot assigned\n\n\nM3A\nNorth York\nParkwoods\n\n\nM4A\nNorth York\nVictoria Village\n\n\nM5A\nDowntown Toronto\nHarbourfront\n\n\nM6A\nN'

The table is found and assigned to the one variable.
It is observed that every rows of the table are within <tr> and </tr>, so to take out the rows of the table, we run the following cell.

Noted that there is a 'next line' ('\n') at the end of each word, so we apply **.split** while assigning the rows to a list.

In [8]:
# Search for all rows from the table using .find_all('tr')
result_table_rows = result_table.find_all('tr')

# Initialize a list 
result_text = []

# Assign the rows to the list
for text_row in result_table_rows:
    result_text.append(text_row.text.split('\n'))

# To check the first 10 elements in the list
for i in range(0,10):
    print(result_text[i])

len(result_text) # check the total number of data

['', 'Postcode', 'Borough', 'Neighborhood', '']
['', 'M1A', 'Not assigned', 'Not assigned', '']
['', 'M2A', 'Not assigned', 'Not assigned', '']
['', 'M3A', 'North York', 'Parkwoods', '']
['', 'M4A', 'North York', 'Victoria Village', '']
['', 'M5A', 'Downtown Toronto', 'Harbourfront', '']
['', 'M6A', 'North York', 'Lawrence Heights', '']
['', 'M6A', 'North York', 'Lawrence Manor', '']
['', 'M7A', 'Downtown Toronto', "Queen's Park", '']
['', 'M8A', 'Not assigned', 'Not assigned', '']


288

The first element is the headings of the 'DataFrame' while the remaining is the values. 

The following commands will sort out the values and assign them accordingly to a DataFrame.

In [9]:
# Assign the data frame headings to a separate list
headings = result_text[0][1:4]
headings[0] = 'PostalCode' # same as the headings given in the assignment
headings

['PostalCode', 'Borough', 'Neighborhood']

- ### Create a new data frame for this table

In [10]:
# Assign data to list
data_element = []
for row in range(1, len(result_text)):
    data_element.append([result_text[row][1], result_text[row][2], result_text[row][3].rstrip('\n')])

# Create DataFrame using the list 
df_toronto = pd.DataFrame(data_element, columns = headings)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


We may ignore the cells without a Borough assigned, i.e. "Not assigned" under Borough column.

There are a total of 77 Boroughs that are 'Not Assigned'

In [11]:
df_toronto.Borough.value_counts()

Not assigned        77
Etobicoke           44
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

- ### Drop cells with a Borough with 'Not assigned'

In [12]:
df_toronto_dropNA = df_toronto[df_toronto.Borough != 'Not assigned']
df_toronto_dropNA.reset_index(drop=True, inplace=True)
df_toronto_dropNA.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


- ### Rename the Neighborhood with the value 'Not assigned' to its Borough name

In [13]:
# Loop to find the value 'Not assigned' in the Neighborhood
for index, data_row in df_toronto_dropNA.iterrows():
    if data_row['Neighborhood'] == 'Not assigned':
        data_row['Neighborhood'] = data_row['Borough']

df_toronto_dropNA.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


- ### Group the neighborhoods of the same borough

In [14]:
# group neighborhoods of the same borough
# df_toronto_grouped = df_toronto_dropNA.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ', '.join(x))
df_toronto_grouped = df_toronto_dropNA.groupby("PostalCode").agg({"Borough":"first", "Neighborhood": ', '.join}).reset_index()
df_toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


- ### Test the grouped and cleaned DataFrame if it is done correctly
  - #### by comparing it with question

In [15]:
# Create a DataFrame for testing
check_df = pd.DataFrame(columns=headings)
check_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for code in check_postalcode:
    check_df = check_df.append(df_toronto_grouped[df_toronto_grouped["PostalCode"] == code])

check_df.reset_index(drop=True)

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


- ### Print the size of the cleaned dataframe

In [16]:
# The shape of the cleaned and grouped DataFrame
df_toronto_grouped.shape

(103, 3)